In [2]:
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BigBirdTokenizer,
    BigBirdModel,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import random

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = nn.BCEWithLogitsLoss(reduction='none')(input, target)
        pt = torch.exp(-ce_loss)
        
        # Focal Loss: alpha * (1-pt)^gamma * log(pt)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

class Config:
    def __init__(self):
        self.model_name = "google/bigbird-roberta-base"

        self.max_length = 4096
        self.batch_size = 2
        self.accumulation_steps = 8
        self.learning_rate = 2e-5 
        self.num_epochs = 50
        self.min_tag_freq = 70

        self.base_dir = "./longformer_project_fixed"
        self.tokenized = f"{self.base_dir}/tokenized"
        self.checkpoints = f"{self.base_dir}/checkpoints"
        self.best_model_path = f"{self.base_dir}/best_model.pt"
        self.mlb_path = f"{self.base_dir}/mlb.pkl"

        os.makedirs(self.base_dir, exist_ok=True)
        os.makedirs(self.tokenized, exist_ok=True)
        os.makedirs(self.checkpoints, exist_ok=True)

config = Config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import pandas as pd
import json
import os
from typing import Dict, List

MAIN_DATASET_FILE = "ao3_dataset.json"
ADD_DATASET_FILE = "ao3_dataset_add.json"
OUTPUT_DATASET_FILE = "ao3_dataset_combined.json" 
FILTERED_DATASET_FILE = "ao3_dataset_filtered_18+.json"

ADULT_TAGS = {
    'rating:explicit', 
    'rating:mature',
    'warning:rape/non-con',
    'anal sex', 
    'blow jobs', 
    'bdsm', 
    'breeding', 
    'creampie', 
    'explicit sexual content',
    'gangbang', 
    'gay sex', 
    'lesbian sex', 
    'mpreg',
    'oral sex',
    'porn', 
    'rape',
    'sex', 
    'smut', 
    'underage sex - freeform',
    'vaginal sex',
    'nipple play',
    'deepthroating',
    'dirty talk',
    'double penetration',
    'size kink'
    
}


def load_dataset(filename: str) -> pd.DataFrame:
    if not os.path.exists(filename):
        print(f"Ошибка: Файл {filename} не найден.")
        return pd.DataFrame()
    try:
        df = pd.read_json(filename)
        print(f"Загружено {len(df)} записей из {filename}.")
        return df
    except Exception as e:
        print(f"Ошибка при чтении файла {filename}: {e}")
        return pd.DataFrame()


df_main = load_dataset(MAIN_DATASET_FILE)
df_add = load_dataset(ADD_DATASET_FILE)
df_combined = pd.concat([df_main, df_add], ignore_index=True)

print(f"\nОбъединено: Итоговый размер датасета: {len(df_combined)} записей.")

if 'work_id' in df_combined.columns:
    df_combined.drop_duplicates(subset=['work_id'], keep='first', inplace=True)
    print(f"После удаления дубликатов: {len(df_combined)} записей.")

df_combined.to_json(OUTPUT_DATASET_FILE, orient='records', indent=2, force_ascii=False)
print(f"Объединенный датасет сохранен в {OUTPUT_DATASET_FILE}.")

def is_adult_content(tags: list) -> bool:
    if not isinstance(tags, list):
        return False
        
    lower_tags = {t.lower().strip() for t in tags}
    
    return bool(lower_tags.intersection(ADULT_TAGS))

df_combined['is_adult'] = df_combined['tags'].apply(is_adult_content)

rows_to_keep_mask = pd.Series(True, index=df_combined.index)

for i in range(len(df_combined)):
    if df_combined.iloc[i]['is_adult'] and (i + 1) % 3 == 0:
        rows_to_keep_mask.iloc[i] = False

df_filtered = df_combined[rows_to_keep_mask].copy()

works_removed = len(df_combined) - len(df_filtered)
print(f"\nФИЛЬТРАЦИЯ 18+:")
print(f"Удалено {works_removed} работ (каждая 3-я 18+ работа).")
print(f"Новый размер датасета: {len(df_filtered)} записей.")

df_filtered.to_json(FILTERED_DATASET_FILE, orient='records', indent=2, force_ascii=False)
print(f"Отфильтрованный датасет сохранен в {FILTERED_DATASET_FILE}.")


def make_full_text(row):
    parts = []

    if isinstance(row.get("title"), str):
        parts.append(f"[TITLE] {row['title']}")

    if isinstance(row.get("summary"), str):
        parts.append(f"[SUMMARY] {row['summary']}")

    if isinstance(row.get("fandoms"), list):
        parts.append("[FANDOMS] " + ", ".join(row['fandoms']))

    if isinstance(row.get("characters"), list):
        parts.append("[CHARACTERS] " + ", ".join(row['characters']))

    if isinstance(row.get("text_sample"), str):
        parts.append("[STORY] " + row["text_sample"])

    return "\n".join(parts)

df_filtered["full_text_input"] = df_filtered.apply(make_full_text, axis=1)

df = df_filtered

print("\nПервые две записи отфильтрованного датасета:")
print(df_filtered.head(2))

Загружено 5167 записей из ao3_dataset.json.
Загружено 2074 записей из ao3_dataset_add.json.

Объединено: Итоговый размер датасета: 7241 записей.
После удаления дубликатов: 7171 записей.
Объединенный датасет сохранен в ao3_dataset_combined.json.

ФИЛЬТРАЦИЯ 18+:
Удалено 1049 работ (каждая 3-я 18+ работа).
Новый размер датасета: 6122 записей.
Отфильтрованный датасет сохранен в ao3_dataset_filtered_18+.json.

Первые две записи отфильтрованного датасета:
    work_id               title  \
0  16793020     Behind the Mask   
1  29134908  Strange Bedfellows   

                                                tags  \
0  [Original Character(s), Space Marines, Aeldari...   
1  [Fluff and Smut, First Time, Interspecies Sex,...   

                 relationships  \
0                           []   
1  [Roboute Guilliman/Yvraine]   

                                          characters  \
0  [Original Space Marine Character(s), Original ...   
1    [Roboute Guilliman, Yvraine (Warhammer 40.000)]   

In [4]:
import pandas as pd
from collections import Counter
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
EXPLICIT_TAG_REPLACEMENTS = {
    'breeding kink': 'breeding',
    'impregnation': 'breeding', 

    'rape': 'rape/non-con elements',
    'dubious consent': 'rape/non-con elements',
    
    'not beta read': None,
    'animated gifs': None,
    
    'anal fingering': 'explicit sexual content',
    'anal sex': 'explicit sexual content',
    'belly bulge': 'explicit sexual content',
    'blow jobs': 'explicit sexual content',
    'come eating': 'explicit sexual content',
    'come inflation': 'explicit sexual content',
    'come swallowing': 'explicit sexual content',
    'creampie': 'explicit sexual content',
    'crying during sex': 'explicit sexual content',
    'cunnilingus': 'explicit sexual content',
    'deepthroating': 'explicit sexual content',
    'dirty talk': 'explicit sexual content',
    'doggy style': 'explicit sexual content',
    'double penetration': 'explicit sexual content',
    'exhibitionism': 'explicit sexual content',
    'face-fucking': 'explicit sexual content',
    'first time': 'explicit sexual content',
    'futanari': 'explicit sexual content',
    'gangbang': 'explicit sexual content',
    'girl penis': 'explicit sexual content',
    'hand jobs': 'explicit sexual content',
    'large breasts': 'explicit sexual content',
    'large cock': 'explicit sexual content',
    'lesbian sex': 'explicit sexual content',
    'loss of virginity': 'explicit sexual content',
    'mating cycles/in heat': 'explicit sexual content',
    'mpreg': 'explicit sexual content',
    'multiple orgasms': 'explicit sexual content',
    'multiple sex positions': 'explicit sexual content',
    'nipple play': 'explicit sexual content',
    'oral sex': 'explicit sexual content',
    'overstimulation': 'explicit sexual content',
    'penis in vagina sex': 'explicit sexual content',
    'public sex': 'explicit sexual content',
    'riding': 'explicit sexual content',
    'rimming': 'explicit sexual content',
    'rough oral sex': 'explicit sexual content',
    'rough sex': 'explicit sexual content',
    'semi-public sex': 'explicit sexual content',
    'sex': 'explicit sexual content',
    'sex toys': 'explicit sexual content',
    'sexual tension': 'explicit sexual content',
    'shameless smut': 'explicit sexual content',
    'squirting and vaginal ejaculation': 'explicit sexual content',
    'underage sex - freeform': 'explicit sexual content',
    'vaginal fingering': 'explicit sexual content',
    'vaginal sex': 'explicit sexual content',
    'voyeurism': 'explicit sexual content',
    
    'plot what plot/porn without plot': 'smut',
    'porn': 'smut',
    'porn with feelings': 'smut',
    'porn with plot': 'smut',
    
    'parent/child incest': 'incest',
    'sibling incest': 'incest',
    
    'bondage': 'bdsm',
    'dom/sub': 'bdsm',
    'dom/sub undertones': 'bdsm',
    'light dom/sub': 'bdsm',
    'humiliation': 'bdsm',
    'praise kink': 'bdsm',
    'forced orgasm': 'bdsm',
    'spanking': 'bdsm',
    
    'bestiality': 'explicit sexual content',
    'size difference': 'explicit sexual content',
    'size kink': 'explicit sexual content',
}

ALL_REPLACEMENTS = {**EXPLICIT_TAG_REPLACEMENTS}


def collect_tags(row):
    t = []
    
    TAG_TO_EXCLUDE = [
        "i'm bad at tagging", 
        'top zhongli (genshin impact)',
        'top zhongli/bottom tartaglia | childe (genshin impact)', 
        'bottom tartaglia | childe (genshin impact)',
        'bottom wise (zenless zone zero)'
    ]
    
    if isinstance(row.get("tags"), list):
        filtered_tags = [tag for tag in row["tags"] if tag.lower().strip() not in TAG_TO_EXCLUDE]
        t.extend([tag.lower().strip() for tag in filtered_tags])
    
    if isinstance(row.get("categories"), list):
        t.extend(["categories:" + r.lower().strip() for r in row["categories"]])
    
    if isinstance(row.get("rating"), str):
        t.append("rating:" + row["rating"].lower().strip())
    
    return t


def normalize_tags(tags_list):
    normalized = set()
    for tag in tags_list:
        new_tag = ALL_REPLACEMENTS.get(tag, tag)
        
        if new_tag is not None:
            normalized.add(new_tag)
            
    return list(normalized)

df["all_tags"] = df.apply(collect_tags, axis=1)

df["all_tags_normalized"] = df["all_tags"].apply(normalize_tags)

all_tags_normalized = []
for lst in df["all_tags_normalized"]:
    all_tags_normalized.extend(lst)

counts = Counter(all_tags_normalized)

common = {t for t, c in counts.items() if c >= config.min_tag_freq}
print("Частых тегов после нормализации:", len(common))

filtered = [
    [t for t in lst if t in common]
    for lst in df["all_tags_normalized"]
]

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(filtered)
pickle.dump(mlb, open(config.mlb_path, "wb"))

config.num_classes = Y.shape[1]
print("Новое количество классов:", config.num_classes)

Частых тегов после нормализации: 84
Новое количество классов: 84


In [5]:
print("Все классы модели:")
for c in mlb.classes_:
    print(f"'{c}',")

Все классы модели:
'aftercare',
'age difference',
'alpha/beta/omega dynamics',
'alternate universe',
'alternate universe - canon divergence',
'alternate universe - modern setting',
'angst',
'angst and hurt/comfort',
'angst with a happy ending',
'banter',
'bdsm',
'begging',
'biting',
'breeding',
'canon compliant',
'canon-typical violence',
'categories:f/f',
'categories:f/m',
'categories:gen',
'categories:m/m',
'categories:multi',
'categories:other',
'character study',
'choking',
'comedy',
'consensual sex',
'crack',
'crack treated seriously',
'crying',
'cuddling & snuggling',
'dead dove: do not eat',
'developing relationship',
'domestic fluff',
'drama',
'drama & romance',
'emotional hurt/comfort',
'established relationship',
'explicit sexual content',
'falling in love',
'first kiss',
'flirting',
'fluff',
'fluff and angst',
'fluff and humor',
'fluff and smut',
'found family',
'friends to lovers',
'friendship',
'getting together',
'grief/mourning',
'happy ending',
'humor',
'hurt/comfort',


In [6]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["full_text_input"].tolist(),
    Y,
    test_size=0.05,
    random_state=42
)

len(train_texts), len(test_texts)

(5815, 307)

In [7]:
from transformers import BigBirdTokenizer 

tokenizer = BigBirdTokenizer.from_pretrained("F:/Politex/ML/3/spiece.model")

def tokenize_split(texts, labels, name, chunk_size=300):
    out_dir = f"{config.tokenized}/{name}"
    os.makedirs(out_dir, exist_ok=True)

    if os.listdir(out_dir):
        print(f"{name}: чанки уже существуют — пропуск.")
        return len(os.listdir(out_dir))

    chunk_id = 0

    for i in range(0, len(texts), chunk_size):
        part = texts[i:i + chunk_size]
        lab = labels[i:i + chunk_size]

        enc = tokenizer(
            list(part),
            padding="max_length",
            truncation=True,
            max_length=config.max_length,
            return_tensors="pt"
        )

        obj = {
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "labels": torch.tensor(lab, dtype=torch.float32)
        }

        torch.save(obj, f"{out_dir}/chunk_{chunk_id}.pt")
        chunk_id += 1

    return chunk_id

train_chunks = tokenize_split(train_texts, train_labels, "train")
test_chunks = tokenize_split(test_texts, test_labels, "test")

train_chunks, test_chunks

F:\Politex\ML\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1964: FutureWarning: Calling BigBirdTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


train: чанки уже существуют — пропуск.
test: чанки уже существуют — пропуск.


(20, 2)

In [8]:
class ChunkedDataset(Dataset):
    def __init__(self, name, num_chunks):
        self.name = name
        self.dir = f"{config.tokenized}/{name}"
        self.num_chunks = num_chunks

        self.sizes = []
        self.cumulative = []
        total = 0

        for i in range(num_chunks):
            d = torch.load(f"{self.dir}/chunk_{i}.pt")
            s = len(d["input_ids"])
            total += s
            self.sizes.append(s)
            self.cumulative.append(total)

        self.total = total
        self.current_chunk = None
        self.current_id = -1

    def __len__(self):
        return self.total

    def load_chunk(self, i):
        if i != self.current_id:
            self.current_chunk = torch.load(f"{self.dir}/chunk_{i}.pt")
            self.current_id = i

    def __getitem__(self, index):
        chunk_idx = next(i for i, v in enumerate(self.cumulative) if index < v)
        local_idx = index if chunk_idx == 0 else index - self.cumulative[chunk_idx - 1]

        self.load_chunk(chunk_idx)
        d = self.current_chunk

        return {
            "input_ids": d["input_ids"][local_idx],
            "attention_mask": d["attention_mask"][local_idx],
            "labels": d["labels"][local_idx]
        }


train_ds = ChunkedDataset("train", train_chunks)
test_ds  = ChunkedDataset("test",  test_chunks)

train_loader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=config.batch_size, shuffle=False)

C:\Temp\ipykernel_11564\517786851.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d = torch.load(f"{self.dir}/chunk_{i}.pt")


In [9]:
class BigBirdForTags(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.body = BigBirdModel.from_pretrained(
            config.model_name,
            weights_only=False
        )

        hidden = self.body.config.hidden_size

        self.norm = nn.LayerNorm(hidden, elementwise_affine=False)

        self.classifier = nn.Sequential(
            nn.Linear(hidden, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def encode(self, input_ids, attention_mask):
        out = self.body(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        cls = out.last_hidden_state[:, 0]
        cls = self.norm(cls)

        return cls

    def forward(self, input_ids, attention_mask, return_embeddings=False):
        cls = self.encode(input_ids, attention_mask)

        if return_embeddings:
            return cls

        return self.classifier(cls)

In [10]:
def evaluate(model, loader):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for batch in loader:
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            y = batch["labels"].cpu().numpy()

            logits = model(ids, mask)
            pr = torch.sigmoid(logits).cpu().numpy()

            preds.append(pr)
            trues.append(y)

    P = np.vstack(preds)
    T = np.vstack(trues)

    P_bin = (P > 0.36).astype(int)

    return {
        "precision": precision_score(T, P_bin, average="micro", zero_division=0),
        "recall": recall_score(T, P_bin, average="micro", zero_division=0),
        "micro_f1": f1_score(T, P_bin, average="micro", zero_division=0),
        "macro_f1": f1_score(T, P_bin, average="macro", zero_division=0)
    }

In [10]:
def extract_epoch(fname):
    return int(fname.replace("epoch_", "").replace(".pt", ""))
    
def train_model():
    model = BigBirdForTags(config.num_classes).to(device)
    
    model.body.gradient_checkpointing_enable()
    model.body.config.use_cache = False

    optimizer = AdamW(model.parameters(), lr=config.learning_rate)
    
    loss_fn = FocalLoss(gamma=2.0, alpha=0.5)
    
    total_steps = (len(train_loader) // config.accumulation_steps) * config.num_epochs

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        int(total_steps * 0.1),
        total_steps
    )

    best_f1 = 0
    start_epoch = 0

    ckpts = [f for f in os.listdir(config.checkpoints) if f.endswith(".pt")]
    if ckpts:
        last = max(ckpts, key=extract_epoch)
        print("Загружаю чекпоинт:", last)
        ckpt = torch.load(f"{config.checkpoints}/{last}", map_location=device)

        model.load_state_dict(ckpt["model"])
        optimizer.load_state_dict(ckpt["optimizer"])
        scheduler.load_state_dict(ckpt["scheduler"])
        best_f1 = ckpt["best_f1"]
        start_epoch = ckpt["epoch"]

    for epoch in range(start_epoch, config.num_epochs):
        print(f"\nЭпоха {epoch+1}/{config.num_epochs}")
        model.train()
        optimizer.zero_grad()

        for step, batch in enumerate(tqdm(train_loader)):
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            y = batch["labels"].to(device)

            logits = model(ids, mask)
            loss = loss_fn(logits, y)
            loss = loss / config.accumulation_steps

            loss.backward()

            if (step + 1) % config.accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

        metrics = evaluate(model, test_loader)
        print(metrics)

        ckpt = {
            "epoch": epoch + 1,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "best_f1": best_f1
        }
        torch.save(ckpt, f"{config.checkpoints}/epoch_{epoch+1}.pt")

        if metrics["micro_f1"] > best_f1:
            best_f1 = metrics["micro_f1"]
            torch.save(model.state_dict(), config.best_model_path)
            print("Лучшая модель обновлена!")

    return model


model = train_model()

⏳ Загружаю чекпоинт: epoch_50.pt


C:\Temp\ipykernel_25332\1715735550.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(f"{config.checkpoints}/{last}", map_location=device)


In [16]:
def load_best_model():
    model = BigBirdForTags(config.num_classes).to(device)
    model.load_state_dict(torch.load(config.best_model_path, map_location=device))
    model.eval()
    return model

model = load_best_model()
mlb = pickle.load(open(config.mlb_path, "rb"))

def predict(text):
    enc = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=config.max_length,
        return_tensors="pt"
    )
    ids = enc["input_ids"].to(device)
    m   = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(ids, m)
        pr = torch.sigmoid(logits).cpu().numpy()[0]

    idx = np.where(pr > 0.40)[0]
    return mlb.classes_[idx], pr[idx]

predict("""[TITLE] Sweetness Can Be a Gift Too, Y'know?
[SUMMARY] Christmas drew ever nearer, and per Dan Heng's suggestion Caelus did a little shopping, if only to avoid the last-minute rush that happened the previous year. Lessons learned.

What he didn't anticipate though was to get a text from a certain Fool, though he certainly wasn't displeased to hear from her again. And with the Astral Express so quiet, it was the perfect time to spend some time together away from prying eyes...
[FANDOMS] 崩坏：星穹铁道 | Honkai: Star Rail (Video Game)
[CHARACTERS] Caelus (Honkai: Star Rail), Sparkle (Honkai: Star Rail)
[STORY] Christmas was a celebration throughout much of the universe, Caelus found out.

The names and subtler details of the festival always changed. Many IPC worlds had the type of Christmas the Astral Express had – big fancy trees, lots of decorations and lights, present-giving and present-receiving; it was the first Christmas he'd had in his new life aboard the Express. Topaz and Aventurine also had such Christmases, though naturally theirs were a bit more extravagant and also dabbled with corporate parties neither wanted but were obligated to attend.

Penacony didn't quite celebrate the season but certainly put up a token effort – enough to say they did but not enough to overhaul things. They sold festive-themed Dream Bubbles, erected a park dedicated to wintery fun and otherwise treated it like a limited time event – because in Penacony that's what it was. A temporary thing that was soon forgotten once the marketing had run its course.

The Xianzhou celebrated it under a different name and with more of a focus on collectivism – on sharing burdens, on forging bonds and on unity against the endless foe that was the Abundance. They gave gifts and whatnot but also helped each other out, like shovelling snow for elders or repairing rooftops, or even just getting shopping for those not able. It was less a need to 'do a good' and more of a 'gift' in its own right. Though from what Sushang told him the more outlanders that came to the Xianzhou Alliance the more it became closer to the IPC's version of Christmas, with a greater focus on presents and festivities.

Belobog was like neither. Being a world of ice and snow they had it in abundance, and the many pine trees that sprouted on the world made for good, quality Christmas trees – a new export, he remembered Topaz mentioning in passing. Moreover there was a dominant theme of sharing; of coming together by the fire and sharing stews, of huddling sharing stores and most dominantly of sharing food. It wasn't as serious an affair as on the Xianzhou, but there was a strong undercurrent of togetherism – of people not only by want but by need coming together to survive and share some ever-needed hope between them.

Of course, Belobog wasn't in such dire straits nowadays but culture didn't change overnight – thus, while Belobog had plenty of stores that sold toys, gifts and even whacky inventions, there was even more food stores. Butchers were aplenty, communal stews were present through multiple places in the city and there was a ball-based game going on in the square; some game between the guards and would-be recruits.

The mood was light-hearted, judging by the cheers and laughter he could hear from the other side of the square – the noise so warm it made him smile, even as his eyes reflected back at him in a shop's window, the frost creeping up the corners but no more. People passed by but none stopped to speak with him, which he was glad for; he just wanted to soak in this pleasant atmosphere. It was a nice change of pace from the rush that often came with the various worlds.

That, and nearly being arrested... thrice.

'Will the next world we visit also involve us getting arrested?' Dryly he thought to himself, shaking his head. 'Maybe I should just dress up as Santa and pretend I'm there to deliver presents.'

Unlikely to work, alas, but he huffed amusedly at the thought all the same. However nice as the atmosphere was he didn't just come here to bask in it – he had shopping to do. Several quick steps brought him to the heavy wood door of the store; the hinge creaking and squeaking as he pushed it inwards, the weight enough to make him grunt but no more. It swung shut slowly when he passed over the threshold and closed behind him with the quiet suck of rubber, though his attention was already on one object in the cluttered, cramped room in particular.

''Is there something I can do for you, sir?'' A youngish voice from behind the counter drew his attention – a woman, probably in her mid-twenties by his judgement.

Caelus hummed, thinking on it for only a moment before plucking a ribbon off of the rack. It was silky and rose red, reminding him vividly of a certain minx that so often made him smile.

''Pela told me you type stuff on these?''

''Oh, you know Pela?'' The woman perked up – only to flush, quickly coughing. ''I-I mean yes, we do. Any message you want can be printed upon it in minutes – do you have one in mind?''

That he did; he'd been musing it ever since Pela had told him about the cosy little store on the main plaza, known for selling knick-knacks and more niche items, usually as gifts.

''Yeah.'' He handed over the ribbon, drawing a breath. ''Type on it... 'My Firework'.''

''Oh that's a sweet one – for a partner, I'm guessing?''

''Mm. How long will it take?'' He inquired, glancing about idly.

''Only a few minutes sir, please feel free to browse until then.''

Caelus agreed, and at her request for payment he foisted over the needed credits – busying himself with looking at the various wares of the store. It wasn't a big place; the ceiling was high but the room was deeper than it was wide. Couple that with shelves lining both walls, the back wall, the front and also through the centre and you barely had space for one person to walk through the store at a time. Yet there was a pleasantness in that cosy vibe; like you could grab a book and just tuck yourself into some corner and read, the daylight coming through the window good enough to read by.

The quiet was soon joined by the dull but quick thunks of a machine. It reminded him almost of a hole-puncher mixed with a typewriter; the dings, clicks and thunks old-fashioned yet weirdly soothing. Mentally he made a note to buy Welt something suitably old-fashioned too. He hadn't asked how old the man was, but going by his experience he guessed several hundred years. Probably. Maybe.

'...would he be upset if I asked? I know Himeko didn't like it when I asked...'

Then again, the mental image of Welt spitting out his tea after being assumed he was centuries old was comical.

A sharp ding drew his attention then; the noise preluding the cashier's voice. ''It's done, sir!''

Meandering away from the row of fine dishes he'd been eyeing Caelus glanced down at the counter, pleasantly surprised. The red ribbon now had two lines of gold to add definition to it, alongside the words 'MY FIREWORK' in bold capitals. He hummed his approval, and when she pulled out a little black box and gently tucked the ribbon into it he could only smile.

''Thanks for the help.''

''It was no trouble sir, have a nice day!''

''Will do.''

Cold air greeted him when he stepped back outside, inhaling a cleansing breath. One more gift gotten, who knows how many more to get. Christmas was still a couple weeks away though so it wasn't a huge rush – small mercies that Dan Heng had convinced him to do shopping early, otherwise it'd be like last year where it had been a last-second rush.

'Hm, maybe I should-'

His thoughts were interrupted by a vibration in his pocket, the sensation catching his attention. Pocketing his gift box he pulled his phone out instead – and one look at the icon of the texter immediately made him open up the DM.

[Come back to the Express, I'm waiting for you~]

That was all the text said, making him raise an eyebrow and fire back a 'whereabouts are you?'. only to receive radio silence. She read it, his phone marked the message as such, but she didn't respond.

'Tease.' He sighed, pocketing his phone.

Then again, his curiosity was too much to ignore. Thus after navigating to a more private alley he tapped into the Trailblaze – and a moment later, he was back aboard the Astral Express.

The change in temperature from mild to cosy was a bit stark but after a shiver or two he got over it, his gaze flitting around the party car. Nobody around, though he did take a moment to appreciate the bevy of fairy lights that lined the top of the train car and the large Christmas tree over in one corner, already hosting a fair few wrapped presents for various members of the crew. He'd already spied a few with his name on it, though he respected the traditions too much to actually feel up the presents to try and gauge what they were.

Typing on his phone again he sent off another text, [I'm aboard, where are you?]

He got a reply in seconds. [Won't you come and search for me?]

Caelus almost – almost – did just that. Until he remembered that Sparkle could be hiding just about anywhere, with her ability to blend in so well – and he last thing he wanted was to barge in on one of his companions in their rooms or elsewhere.

Thus, he fired off his best guess. [You're in my room aren't you?]

A pouty Fu Xuan emoji was sent back at him, all but confirming his suspicions. He chuckled under his breath and made his way towards the stairs; deviating from his path only long enough to deposit his newly-obtained gift under the tree before ascending the stairs to his personal quarters. He could hear no music nor any other suspicious noises through the thick walls, though he still opened the door slowly lest a surprise confetti cannon be waiting for him on the other side.

None were, though his suspicions were not so easily quelled. Carefully he crept inside, one step after the other, fully prepared to raise his arms to block a sudden high-velocity plushie-

''Boo!''

-and yet he still flinched when Sparkle jumpscared him – from behind. He stifled a curse and groaned it out, turning around to face the minx standing behind him. One who giggled not a moment later, elation lacing her voice and her eyes full of teasing mirth. It was only when he thought to look up that he noticed the slight swing of the hanging light.

''You were hanging onto it, weren't you?''

''Mm, maybe, maybe~'' Sparkle tittered, arms behind her back and smirk decidedly sexy. ''About time you came home, Gray Hair~''

''I told you I was gonna be busy shopping.'' Caelus huffed, though it lacked much heat. ''Nice outfit by the way.''

Clearly he wasn't the only one in the festive mood based on her outfit alone. A thick yet snug red dress hugged Sparkle's figure closely, tied by a belt at the waist and reaching no further than her thighs whilst remaining completely shoulderless – leaving those slender, creamy shoulders of hers on full display, covered only by the small bell neckpiece she wore. It was that same bell, glinting in the overheard light, that lured his eyes down to the fluffy white trim of her dress, unable to not notice how emphasised her bust was in such thick clothing, nor how pretty her hands looked when poking out from the detached sleeves of her attire, nails painted red and fingers wiggling in delight.

Sparkle preened under the praise, giggling as she brought those same fingers up and splayed them across her mouth, doing little to hide her cheeky grin. ''Aw, thanks~ I knew you'd like it. The IPC has a surprisingly wide selection of these outfits for sale y'know? They do everything from fancy party dresses to pyjamas, to shorts and tops – to even Santa bikinis, how wild is that~?''

The teasing lilt to Sparkle's tone only fuelled the ideas her words put into his mind; warmth quick to colour his cheeks. ''Er, right... sounds marketable, I guess...''

''Hm? No pleas for me to wear those outfits? I'm hurt, Gray Hair~'' Sparkle pouted dramatically.

Caelus shook his head though, ''I like the one you're wearing right now. Really highlights your cuteness.''

The compliment was easy to make, and he was rewarded as Sparkle blushed – a light dusting of pink and no more, but she preened all the same; giggling. ''Well well, look who's a charmer today~ Guess I put you in a good mood, hm?''

''Always.'' Again he saw her teasing mask crack – flusterment flashing in her eyes before she covered it with a cheeky gleam.

Still, he saw, and the idea that his words could make her heart skip a beat just emboldened him to tease her more.

Sparkle snapped her fingers then. ''Oh, would you look at that Gray Hair~''

A point up lured his gaze there – and there, where they had only been empty ceiling before, now hung a mistletoe.

''Wow, I wonder when that got there. You're so mysterious, manifesting something like that!'' Sparkle exaggeratedly put a hand on her chest, but her grin gave her away. ''Since it's tradition, I guess we have no choice, right~?''

Caelus's lips twitched up, ''Y'know you could just ask for a kiss?''

Sparkle pouted. ''Now where's the fun in that?''

In response Caelus took Sparkle by the waist and pulled her in – their lips meeting in a sudden, but far from unwanted kiss. The familiar contours of Sparkle's lips against his own was heavenly; the softness, the slight plumpness, the way they parted as she moaned into his mouth and how they conformed against his own as best as possible, deepening the kiss to intoxicating levels. He drew it out for as long as possible, their surroundings fading from sight as he just focused on her and her alone.

When the need to breathe drew him back, a lungful satiating his need – and then he stole another kiss. This time Sparkle moaned in surprise but didn't hesitate to prod his teeth with her tongue, enticing his own to slip out and meet hers – their tongues wrestling in a slow, messy tangling of flesh, spit swapped and her groan spilling into his mouth. His hands dipped down her waist and he grabbed her ass rather audaciously; pulling her closer still and earning a drawled coo from his partner, her tongue still tangling with his.

Until with a hearty exhale she broke it off – pulling back with a wicked gleam in her eyes and a sweetness in her smile as she took his hand. Caelus didn't fight her pull, following only a step behind as Sparkle led him around the corner of his room and straight to the bed. She walked backwards, calves hitting the edge and sending her falling back – and with hand in his she pulled him down with her.

He caught himself lest they butt heads but Sparkle just giggled, another softer tug pulling him down until her bottom lip touched his. The ensuing kiss was deep, loving and rich in affection; said affection spelled out to him as her dainty fingers traced love hearts across his clothed pectorals, her other hand cupping his nape only to draw down his back, pressing jacket to his skin and tracing around to his front, working its way down to his belt.

Caelus didn't stop her; a low groan escaping him as her other hand joined it. Together they undid his belt in seconds, prying it open and unbuttoning his pants enough that she could fish his cock out – cooing against his lips as she caught his cock in her soft palm and curled her slender fingers around it. Just being in Sparkle's hand was erotic but to feel her squeeze added an extra layer to it; how gentle she was as she moulded her hand to his girth, caressing him so sensually he could only groan in bewitched bliss.

Sparkle giggled then, drawing back from the kiss to smile at him – eyes narrowed with mirth, cheeks tinted pink and smile tamer than past smirks, yet no less teasing. ''You're making an awful lot of cute noises Gray Hair~ You must've really wanted to see me, huh~''

''You invited me back though?''

''Details, details~'' She waved off; tickling her fingers up the underside of his manhood and drawing a circle around his tip. ''Mm... it's so big today too~''

He knew it was just a tease but it spoke to how good Sparkle was at it that he felt himself throb, his girth aching against her warm palm – wanting to be somewhere warmer, more intimate. As if sensing that thought Sparkle giggled and let go of his cock to cup his cheek instead; tugging him down and sealing her lips against his in a deep, riveting kiss, their lips remaining connected even as Caelus himself caressed her side.

And when the kiss was broken he shifted back. Sparkle made a noise of surprise, brows rising – and her blush darkened as he got between her legs. Despite her flusterment she didn't stop him as he leaned down and pressed his lips against the warm flesh of her thigh; eliciting a shyer coo from his lover even as he meticulously kissed his way along her supple flesh, taking the time to admire her thighs after she so frequently fought alongside him.

There was only so many high-kicks and panty flashes a man could take, after all, and her flexibility was matched only by how lovely her thighs were.

''Ooh, you're really in the mood huh~''

Sparkle's tease went unheeded, lust and love swirling around Caelus's mind as he swapped over to her left thigh, his fingers dimpling her flesh before he chained kisses up it – working his way further and further up, his nose brushing against the fluffy trim of her dress and hiking it up. A tug from Sparkle pulled to out of the way entirely, allowing him to see the white panties she was wearing beneath, the front bearing a little red bow as if an invitation to be unwrapped.

Caelus gladly did so; hooking his fingers into her panties and prying them down; wasting no time in planting a kiss upon her groin. Sparkle moaned, a little shiver vibrating across her skin as he drew her panties down off her legs and let his breath tickle her nether lips – preluding the quick, lashing lick that made Sparkle mewl, her back lifting away from the bed in pleasure. Caelus grabbed her ass in response, lifting it too as as his tongue teased its way along her slit much slower than before; teasing a husky whine out of his lover.

''Gray Hair~'' She whined out his nickname.

It sent a shiver roiling through him – the seductiveness, the horniness that dripped off of her tongue and slithered into his ears. His cock ached and he had to resist the desire to stroke himself off; busying his hands with squeezing her thighs before pressing his tongue into her – curling and flicking it against her entrance in a slow, horny tonguing that made Sparkle whimper, her toes curling in her shoes and her legs rising up higher and higher with every flick of his tongue.

He eased her back down onto the bed though, freeing his right hand and allowing him to spread her petals with two fingers; his tongue lashing at her quivering pink entrance. The sweet whine she made engrained itself into his mind and without thinking twice he let his tongue loose on her more; longer, slower licks teasing a spasm through Sparkle – her lips pursing and quenching her moan into something shyer, the pleasure she felt barely disguised.

Tempting as it was though to lay there and eat her out utter she was dripping and whimpering for more her brief handjob had left him aching. It was that same ache that made him pull away after one final lick; the taste of her on his tongue as he crawled up Sparkle's front. She snatched him by the collar of his shirt and pulled him in the moment he was close, pressing her lips against his and pushing her tongue into his mouth – shameless as she wrestled her tongue with his, getting a taste of herself in the process.

''Mm- haah~'' She exhaled as they broke apart, a little grin curling at her lips. ''Wanna do it now, hm~?''

''Duh.'' Though his tone was flippant the way he caressed her cheek wasn't – bringing a rosier blush to Sparkle's already-flushed cheeks, her smirk turning into a softer, yet no less playful smile.

Gently Caelus eased himself between her legs fully, one hand supporting his weight whilst his other ran along her thigh, giving it one last squeeze before grasping his member. Sparkle crooned and laid back, legs spreading wide and leaving nothing to stop him from rubbing his cock-head along her wet, dripping slit, the sticky sensation enough to elicit a low groan from him. The primal temptation to ram himself inside, to hilt in her dripping depths, it lurked just beneath his skin; begging for him to claim Sparkle like a beast in heat.

He resisted, however. He teased his tip along her slit, dipping in no more than the tip before 'accidentally' missing, the teasing act drawing a soft groan out of Sparkle. She pursed her lips, eyelashes fluttering – and another mewl left her lips as he feigned missing, edging her with the constant, gentle stimulation.

''Gray Hair~'' Soon she whined, pouting. ''Don't make me wait...''

With a ragged breath Caelus rubbed his tip between her folds, the contact making him ache more even as he spoke, ''Then, mn, say my name...''

Sparkle's eyes widened before she looked aside – genuine, endearing flusterment colouring her cheeks even as she looked aside. ''Fine, then...''

''Put it in, Caelus...''

The demure way she breathed his name out, the subtle tremor of need, the undercurrent of want – t was all he needed to hear. He pressed himself into her, hips rolling forth and delving his tip into her wet, welcoming pussy – the sensation as rapturous as all times before. Sparkle arched her back with a gentle cry and without thinking he pinned her wrists to the bed; groaning as he sheathed himself in deeper too, inch after inch of him fitting inside of her slick, squeezing tunnel that moulded itself to his girth.

He pulled out with similar slowness, intimately feeling her wet walls clasp around his shaft as if begging for him to not leave; a sentiment that made him sheath himself right back in her – groaning with her as he filled her up that little bit more, her snug sex smothering his shaft. Any discomfort Sparkle felt fled quickly too, as her arms slid down in his grip until her palms met his; gripping his hands with a mischievous little giggle.

''Make a mess of me, darling~'' The sing-song tone accentuated her devious smile perfectly.

And enticed Caelus to move. Despite the urge within him to be rough he quelled it; shuddering as he pulled partway out and pressed himself deeply into Sparkle. The husky moan he fucked past her lips was pornographic in quality and he revelled in that sound; squeezing her hands in his as he pumped himself into her, his thrusts slow but oh so firm and deep, nearly grinding him into her innermost parts and making sure Sparkle could feel him stretch her out fully with his girth.

The musical moan tinged with a whimper assured him she did. Though she didn't lose her cheeky look the way her eyes screwed shut, how her nails dug between his nails, how her thighs rode up his hips – it was all so intoxicating that he darted down and stole a kiss, muffling Sparkle's horny whine as she reciprocated wholeheartedly, the meshing of lips enamouring and only made hotter as he slowly pumped himself into her, feeling her depths clench around him in a strangling, smothering grip.

''Mmph, mm- ah~'' Sparkle brushed her lips against his own, giggling. ''You're reaching so deep, it feels like you're just waiting for the chance to fill me up~''

''Want me to pull out when it's time?'' Caelus murmured back, curious.

Sparkle cooed, ''Though it feels nice to have you shoot it on my stomach, it'd make a big mess like this, don't you think so? It'd be easier to clean up if you finish inside~''

''And riskier too.''

''What's life without a little risk~?'' The Fool crooned sweetly, devious. ''Or are you afraid of knocking me up~?''

The unabashed way she said it made him shudder, his dick twitching inside of her snug pussy, encouraging him to pump into her faster even as he spoke. ''Mn, I'm just worried I won't be able to stop.''

''Ooh, how naughty~'' Sparkle teased, gripping his hands once more before letting go. ''Let's see if you can really ruin me~''

Some small part of his mind suggested that he should be responsible and pull out, that he shouldn't let lust dictate his actions – but any and all such thoughts were overpowered by the rampant horniness that coursed through him, egged on by Sparkle's cheeky grin. Leaning back on his knees he slowed his thrusts only momentarily so he could grasp her delectable thighs and lift them up, her heels resting against his shoulders.

And Sparkle moaned as he leaned over her, his hands folding her legs over herself in a mating press. Not a moment later Caelus began to move – rougher, faster thrusts burying him between her petals and piping a sharp mewl out of Sparkle, the sinful smack of flesh quick to fill his bedroom as he hilted himself in her welcoming depths. The heat of her skin against his palms was nothing compared to the inner heat clasping around his dick, his length retreating only momentarily before he sheathed it back inside of her dripping tunnel, the friction forcing him to bite his bottom lip.

Sparkle had no such compunctions. ''Mm~! Like that Cae, mess me u-up~''

The slight stutter turned him on more than he wanted to admit, his grip on her flesh tightening and his hips jolting with a sudden buck – fucking a squeal past Sparkle's lips. He swore with her, eyes shutting as he slammed himself into her, her sopping pussy perfect as he hilted inside of it; every impact sending a jolt recoiling through them both, pleasure pervading every aspect of his mind and leaving him able only to keep moving, lest he stop and unceremoniously climax inside of her.

A task made ever harder by the sight of Sparkle with her legs up and pussy bare to him as he sheathed himself inside of her, her gleaming folds brushing against his intruding member with every pump, her flesh gripping him so lovingly as he nearly pulled out only to rut right back into her. The way her creamy thighs were lit by the overhead light just made them seem ever softer, his hands sliding down to grope them even as he pounded himself into her tight hole, the rapturous pleasure bringing him to a panting, twitching peak.

''Sparkle, 'm gonna cum...''

''Ah, let it out Caelus, let it all out~!''

The elated cry, the elated plea – it was too much, his hips slammed forth like a jackhammer and made Sparkle squeal-!

And with a shout he slammed into Sparkle and came, erupting inside of his lover with a ragged gasp. Her pussy clamped like a vice and he bucked his hips madly, cumming senselessly within her and painting her walls a sticky white with his seed; said seed felt as he kept crudely moving inside of her, churning their mixed fluids around her pussy as he repeatedly hilted up against her cervix. Dimly he could feel her spasm beneath him, her cry lost to the deafening roar of pleasure in his ears as he moved, the feel of her warm thighs in his hands and the tightness around his cock the only things grounding him to reality.

Until all at once reality came rushing back in. Overstimulation assaulted his nerves and he set his jaw into a tight line, groaning deeply as he pressed himself deep into Sparkle and stayed there – grinding into her and no more. Sparkle moaned in reply – voice light, shaky and breathless, drawing his eyes up to her face.

She was flushed scarlet, her eyes were half-lidded and the grin on her lips was sexy as hell. Doubly so when she opened up her arms to him, enticing Caelus to release her legs and lay atop her more naturally – their lips meeting in a deep, intoxicating kiss that left them both moaning, his hips feebly rocking and grinding him into her cum-filled pussy.

''Mm- hmhmhm~'' She giggled into the kiss, one hand running through his hair as they parted. ''You must've been super backed-up, huh~? Or did the risk of getting me pregnant really turn you on that much~?''

''Both.'' Caelus admitted, earning another elated titter. ''I mean, you'd make a good mom.''

''Aw, you're so sweet~'' She pecked him on the lips, arms wrapping around him-

-before in a feat of strength, she rolled them over.

''Then we better do it again to be sure, right~?'' Sparkle crooned.

Caelus flushed, both from her words and from suddenly finding himself under her – laid back against the bed with the teasing minx atop him. Sparkle giggled at his reaction and shifted slightly, his dick having slipped out in the movement and enticing her to curl her fingers around his length, stroking him off and bringing him back to full hardness easily, such was her charm. It also didn't hurt that she pulled her dress up and laid her loins bare, the sight of her cum-leaking pussy a turn-on for him.

A sight Sparkle let him enjoy as she shuffled closer, positioning herself above his length once more. Tease that she was though Sparkle didn't let him in right away, instead lowering herself and rubbing along his tip, her sopping pussy rubbing against his sensitive tip and knocking a moan out of his throat. On instinct alone he grabbed her hips but didn't pull, not yet; simply holding her even as she rocked back-and-forth, his tip nearly dipping in but not quite, the temptuous, syrupy heat enamouring-

''Mm~!''

-and then she dropped down. Wet, hot, tight; the words engrained themselves into his mind as Sparkle took his full length inside right off the bat, her deep moan sending a shudder roiling through him. He dug his fingers into her supple hips and groaned with her as she rose back up, her pussy clinging onto his length as if begging for him to come back in – her walls still slick and sticky with their mixed fluids.

Making it feel even hotter when she impaled herself back onto him. The velvety grip around his dick spasmed through his groin and unconsciously he ran his hands up her hips, palming her fine figure all the way up to her waist before bringing his hands back down, squeezing at her thighs. Sparkle tittered at that and grinded herself down onto him in response, her pussy tightening around him as she deliberately tensed her pelvic muscles, wringing grunts out of his throat and teasing mewls out of her own.

''You're still so twitchy, hehe. I guess the thought of cumming inside me is too exciting for you, huh~'' Sparkle breathily teased, winking down at him.

Enticing Caelus to push himself upright. Sparkle barely had time to coo before their lips met – a deep, messy sort of kiss that left her whining, her chin lifting as he took control of the kiss. Her bounces slowed and for a moment they just sat there kissing, their bodies connected and skin contact plentiful.

He broke off the kiss first, losing himself in Sparkle's pretty eyes – before she looked away, blushing. It was impossible to tell whether she was being intentionally cute or that was just her charm, but nonetheless he gripped her right hip and bucked his own, fucking a yelp out of Sparkle as he so suddenly rutted up into her. It stirred her out of her flusterment though, a familiar grin curling at her lips once more.

''Bad boy~'' She crooned, hands on his shoulders.

Right before she pushed him flat on his back. Caelus had no chance to move though before Sparkle did so first – a sharp, elated moan filling his bedroom as she rose up and impaled herself back down, bouncing on him once more and doing so with such enthusiasm he could do naught but groan with her. His fingers once more found purchase on her thighs but he made no move to slow her down; simply caressing them as his lover took him deep inside of her over and over again.

The sinful smacks of flesh were made wet and sticky by their mixed fluids; said fluids oozing down his shaft and squelching when Sparkle grinded herself down against the base of his shaft. Still sensitive from his previous orgasm Caelus could only grunt and hold her thighs as she moved, the tight grip around his cock euphoric and sapping him of breath, every undulating motion threatening to be the one that pushed him over the edge.

Still he endured, grunting between panting breaths, his cock twitching up against her innermost parts – his eyes briefly on their connected loins as she dropped down his length, his gleaming length sinking into her wet core, before glancing up as Sparkle leaned over him more. She giggled breathily; pink eyes lidded and moans soft and sweet, her noises of pleasure only bringing him that much closer to his climax.

''Sparkle, gonna cum again...'' He breathed out in warning.

He felt her tighten around him. ''L-Let it out, Gray Hair~''

Caelus screwed his eyes shut, groaning as he held her waist-

-when Sparkle brought her lips to his ear.

''Get me pregnant~''

That did it. His fumbling hands latched onto her hips as she bounced, her walls clenching around his swollen member – and with a shout Caelus erupted inside of Sparkle, throwing his head back into the quilt as he came inside of her once more. The minx squealed in delight, her voice tapering with pleasure even as she furiously grinded herself onto his twitching member; extracting spurt after spurt of liquid heat from his overstimulated member; his hips bucking as if to help ejaculate it deep into her womb.

Sparkle moaned at that, her grinding slowing even as her insides kept milking him; fatigue apparent in her motions and affirmed by her softer mewl. Still she kept moving even as their orgasms waned; wringing out every last drop of nut before finally flopping against his chest with a delighted mewl; snuggling herself into the crook of his neck without delay.

Caelus held her close, still partly in a daze and shivering from the feeling of her breath on his neck – the quick, ticklish exhales oddly pleasant.

''...hey, Sparkle...?''

''Mm...?''

''Wanna join us for Christmas, this year?'' Caelus offered and felt her stiffen, surprise clear. ''Before you ask, I checked with Himeko a couple of days ago and she'd be fine with it, provided you 'toned down the pranks', in her words.''

Sparkle hummed – a longer, deeper hum as she nestled into his shoulder, clearly thinking it over. He didn't rush her, content to just cuddle the Santa-dressed girl close, their breathing growing steadier by the minute.

Until soon, Sparkle offered her answer.

Namely in the form of a kiss to his cheek.

''Sure! After all, I better get to know your 'family' more before the baby comes~''

The fiery blush on his face was joined by Sparkle's laughter, elated and eternally sweet.""")

C:\Temp\ipykernel_11564\2497847959.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(config.best_model_path, map_location=device))


(array(['categories:gen', 'fluff', 'rating:general audiences',
        'rating:teen and up audiences'], dtype=object),
 array([0.719698  , 0.4209698 , 0.64338166, 0.4753696 ], dtype=float32))

In [22]:
# =============================
# Часть 10. Инференс
# =============================

def load_best_model():
    model = BigBirdForTags(config.num_classes).to(device)
    model.load_state_dict(torch.load(config.best_model_path, map_location=device))
    model.eval()
    return model

model = load_best_model()
mlb = pickle.load(open(config.mlb_path, "rb"))

def predict(text):
    enc = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=config.max_length,
        return_tensors="pt"
    )
    ids = enc["input_ids"].to(device)
    m   = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(ids, m)
        pr = torch.sigmoid(logits).cpu().numpy()[0]

    idx = np.where(pr > 0.30)[0]
    return mlb.classes_[idx], pr[idx]

predict("""[TITLE] Warmth of Wangsheng
[SUMMARY] Hu Tao wasn't exactly sure how this had happened. One moment she'd been walking through Wuwang Hill, helping some spirits, and the next she'd suddenly tripped on a tree root and ended up rolling all the way down the slope. No biggie, maybe - if she hadn't sprained her ankle really badly on the way down.

And you know, Wuwang Hill can be pretty cold at night.

--

OR: In which Hu Tao rolls down a mountain, sprains her ankle, and is stuck shivering in the cold of Wuwang Hill - until her consultant helps her out.
[FANDOMS]  原神 | Genshin Impact (Video Game)
[CHARACTERS] Zhongli (Genshin Impact), Hu Tao (Genshin Impact)
[STORY] Hu Tao wasn’t exactly sure how this had happened. She’d navigated Wuwang Hill dozens of times in her life - she should’ve known the landscape like the back of her hand. It was honestly kind of embarrassing, as the 77th Director of the Wangsheng Funeral Parlor.

Still...she supposed accidents could still happen?

Hu Tao groaned, pushing herself onto bruised elbows. Wuwang Hill seemed to unfold around her in a blur of trees and shadow, strange and unrecognizable shapes, and she blinked her eyes in an attempt to clear them of the haze. Her head pounded, and her body seemed to throb and ache all over. But she supposed these were just the natural consequences of having rolled down a mountain.

The assignment was supposed to be a simple one - help Old Mr. Tang move on. The old man had been lingering in the Harbor for years, waiting patiently in limbo, playing guardian to a wife he’d left behind 3 years before. When Mrs. Tang finally passed on 2 days ago, he was thrilled at the prospect of seeing her again - until he’d navigated the entirety of Liyue and the Border and still found no sign of her spirit.

Poor Old Mr. Tang had dropped into a pleading fit by the parlor, begging Hu Tao to help find her.

“I’ve been waiting for her for years,” he’d sobbed, a real and proper mess, “Where could she have gone?”

So Hu Tao had helped him look through Liyue again, scout through their favorite places, and finally bring him back to the Border to see if he’d somehow missed her amongst the spirits. When even Hu Tao’s keen eye for detail had failed to pluck her out, Old Mr. Tang had been forced to confront the reality that, unlike him, his wife had chosen to forgo all worldly regrets and pass on immediately to the other side.

“But why?” Mr. Tang had said, his face sagging with disappointment. “I’d waited for her for so long. Why wouldn’t she stay to speak a few words of farewell to me?”

“Because sometimes, spirits are happy enough to pass on straight away. To forego limbo and mortal attachments, and choose peace on the other side,” Hu Tao had told him, smiling. “Isn’t that what you want for your wife, Mr. Tang? To know that she died without regrets?”

In the end, Hu Tao’s words had gotten through to him - and he’d faded just as all spirits did when their worldly ties were done.

Satisfied, Hu Tao had left the Border in relatively good spirits - humming merrily through Wuwang Hill with little light besides the glow of butterflies. She had no real fear navigating the woods at night - she knew this mountain like the back of her hand! Or...at least she thought she did - until her foot had snagged on a protruding tree root and sent her sprawling down the slope.

Now, Hu Tao plucked another leaf from her hair and tried to will away some of the aching in her head. And her elbows. And...basically everywhere. “...oof,” she muttered, chuckling to herself. “Well, that sucked.”

For a moment, her vision glowed a hot and fiery red - and out popped the round white shape of her familiar ghostly friend. It whirled around her, murmuring worriedly. But Hu Tao just tsk-ed and waved it off.

“...hey, don’t worry about it! I’m fine.” As if to demonstrate this, Hu Tao shoved herself onto her knees, preparing herself for the long trudge back. “...though, for the record? Rolling down a mountain is definitely not fun. Let’s just make our way back to the Parlor, now shall we-”

Smiling, Hu Tao brushed dirt from her clothes and stood up - or, at the very least, she tried to. But once she so much as put pressure on her right foot, she was immediately assailed by an intense pain that sent her tumbling right back down onto the grass. She cringed, blinking back tears, until the waves of agony ebbed.

Oh.

Oh no.

“...uh oh,” she muttered to herself, feeling the realization dawn. She shoved herself onto her elbows again and looked back - but muffled as her ankle was beneath two layers of sock and shoe, she couldn’t be too certain what the damage was. Was it a sprain? A break? Either way, it hurt like hell.

Her ghost whirled close, its usual smile pulled low into a worried frown. Hu Tao shot it a look.

“Crap, I know, I know. It’s fine though! If I just...hobble a little, I can probably still walk-” To test this, Hu Tao braced herself on a nearby tree and began, very carefully, easing herself back up. She took an experimental hop on her left foot - okay. So far so good. Then slowly, very slowly, she placed her right foot tenderly back onto the ground to test its weight-

Suddenly, the pain flared up so badly Hu Tao felt herself falling back onto the ground. Okay, shit. Shit. She couldn’t walk. Nope. She still didn’t know if it was straight-up broken or just a really, really bad sprain, but it hurt and shit walking was definitely off the table. She was, frankly, in very big trouble.

“...okay,” she said to herself, blinking back tears. “Okay, okay. This is fine. I can just, uh, I can-” What? What could she do? She was stuck in Wuwang Hill, alone, with a very badly hurt foot and no way to walk back home. And it was dark. And...getting kind of cold.

Hu Tao breathed out a shuddering breath. Beside her, her ghost friend drifted close, murmuring worriedly to itself.

“...yeah. Yeah, I think you might be right,” Hu Tao muttered to it quietly. “...I’m screwed.”

*

At the 15 minute mark, things weren’t too bad.

At around 30 minutes, she’d taken to writing poetry in her head.

At the first hour, she’d begun reciting said poetry to her ghost friend; interjecting here and there with questions like what do you think? Do you think I should use this phrase instead of that? And then not waiting for an answer.

At the one and a half hour mark, she’d started igniting the grass next to her to warm her from the night - and almost got burned for her trouble.

At the second hour, though...things were beginning to wear on her.

“...silly-churl, billy-churl, silly-billy hilichurl…” Hu Tao hummed, eyelids fluttering. Her voice was a weak wisp, warmed by the fading embers of her usual cheer. “...frilly-churl, willy-churl, frilly-willy hilichurl...hmm…”

Hu Tao yawned, trying to suppress another shiver. She forgot how cold Wuwang Hill could get at night - then again, she supposed there was no need to fear the chill when you were always sprightly and bouncing. Stuck as she was at the base of the hill, with her foot still throbbing and her face pressed against the damp grass, it seemed the mountain was finally beginning to get to her - and dang. It sucked.

“...ai-ya,” she mumbled, trying to fight back the edges of sleep. “...hey, Boo - you know how I said rolling down a mountain ‘wasn’t fun’? I’m amending that right now. It’s absolute garbage.”

Her ghost frowned at her, waving its arms in consternation.

Hu Tao sighed. She knew her ‘ghost’ wasn’t actually a spirit - it was just an extension of her Vision, a culmination of her Pyro abilities breathed fitting form to it's wielder. Still...she wasn’t in the mood to pocket it, at least not right now. Loathe as she was to admit it, she could use a little company.

Dang, she was tired. And cold. And miserable.

A sprained(?) ankle sucked.

“...silly-churl, billy-churl, silly-willy hilichurl…” she sang anew, ringed around a circle of scorched grass. She ignited a small ember in her hand, cupping it in her palms, hoping the flame would shield her from the growing chill of the night. “...frilly-churl, willy-churl, frilly-willy hilichurl…”

Her voice trailed off. She really was quite sleepy...she supposed there was nothing wrong with her taking a nap?

She dropped her hands, letting the ember die out in a gust. Above her, fringed by the canopy of trees, she could see the night sky glittering - a deep navy canvas awash with endless stars. If she had more of her strength, maybe she would write a poem.

She thought back to the Parlor. About her undertakers, her consultants. About Zhongli. Have they noticed her absence? Would they look for her?

…or maybe Zhongli was still nursing a grudge from her having put a Jueyun Chili in his tea the day before. Still, it'd be nice if he came anyway.

Rolling to her side, the young girl hugged her hat to her chest and let her eyes flutter shut. The night was growing colder, and she was too sleepy to conjure up more flame.

...yeah.

A nap shouldn’t hurt.

*

10 o clock.

Zhongli frowned up at the clock, watching the time whittle slowly away. He'd been waiting at his desk for the better part of an hour, anticipating Hu Tao's appearance - but dusk was deepening, and the Director was still curiously absent.

Hmm, strange. Director Hu was certainly lackadaisical about most things, but the one thing she always took seriously were the workings of the Parlor. They'd set up this meeting specifically so Zhongli could present her the report regarding the new undertakers - so it was strange of her to forgo it, even if it meant irking him.

Zhongli leaned back in his chair, looking back at the clock. A strange emotion had begun settling in his stomach, deepening with every minute that ticked slowly past.

Sighing, Zhongli pushed back his chair and stood up, straightening his lapels. Sweeping out of his office, he pushed open the front door of the Parlor and found the Ferrylady at her post, looking worriedly into the night.

"Pardon me," he greeted politely. "Have you seen Director Hu, by any chance?"

"...ah, Mr. Zhongli," the Ferrylady turned to him and dipped her head, a courteous greeting. "...it's a curious thing. I believe Director Hu set off earlier to fulfill some business at Wuwang Hill, but she's yet to return."

Zhongli frowned. "How long ago was this?"

"About 4 hours ago, I believe." The Ferrylady furrowed her brows, clearly concerned. "...I was given word that she'd be back in time for her meeting with you, but she hasn't yet."

Zhongli kept his face schooled, but that didn't stop an unpleasant feeling from worming its way into his chest. He knew Director Hu was eccentric in her habits - but this particular disappearing act felt a bit more…out of place than most. It left Zhongli with an odd sense of dread, and he didn't like it.

Wuwang Hill… she must have been settling problems of the more spiritual nature, then; around the site of the Border. Inwardly, Zhongli sighed - that was a long journey from the Harbor, to check up on the Director. But if she had somehow ran into a bout of trouble…

"...if that's the case, I will be out for a bit to check up on her, then. Could you please mind the Parlor in my absence?" he asked.

The Ferrylady nodded, smiling faintly despite her worry.

"...of course, Mr. Zhongli. Please, have a safe journey."

Zhongli nodded in his thanks. Adjusting his suit, he wended through the streets of Liyue to the distant peak of Wuwang.

*

"...irector Hu?"

Hu Tao stirred awake, her head cushioned against the grass. For a moment, she wondered if she was dreaming - but the coldness of Wuwang Hill and the pain in her ankle was quick to shatter that illusion.

She knew that voice. She knew.

"Director Hu!"

Footsteps, crunching closer. Hu Tao shrugged the grogginess from her head and looked back, squinting into the dark. Her head was fuzzy, her eyes blurred. She wasn't sure how long she was asleep, huddled small amidst the cold - but the lethargy in her limbs and the cotton in her head told her all she needed to know.

Someone was approaching - no, running towards her. Dark hair. Black suit. Amber eyes.

I know you.

"...Zhongli?"

"Director Hu. What have you been doing here this whole time?" Her consultant neared, dropping to his knees beside her. Through glassy eyes, Hu Tao could see the regular sternness of his expression; its edges made soft by some other, extraneous emotion. Worry. Was it worry? "...you're shivering. Are you hurt?"

Hu Tao's head dropped back onto the grass. She was so tired, tired and cold. "...my ankle hurts."

Zhongli's eyes quickly cut to her leg. "Your ankle?"

"Yeah..." Hu Tao hugged her hat close to her chest, in a way she hadn't done since she was a child. Yeye, can I wear your hat? Isn't it too big for you, little Butterfly? "I tripped and fell down the slope, and now it really hurts."

Zhongli frowned. "...may I examine it?"

Hu Tao gave a weak nod.

Gingerly, Zhongli braced a palm against her leg - and then winced.

"...you're freezing," he said plainly. "Wuwang Hill tends to get colder in the night, and you've been stuck here for hours. Didn't you bring a coat?"

Despite the situation, Hu Tao still managed a weak laugh. "...what are you, my dad?" she chuckled, a soft sound that dissolved into a shiver. "...no. Sorry."

At her apology, Zhongli's concerned frown only deepened. Rather hurriedly, her consultant shrugged off his coat and draped it over her, and ensconced as she became in that oversized jacket, she felt marginally more warm.

"Remember then. Next time," he said gently, before moving his attention back to the ankle. Hu Tao felt him roll down her sock, his hands infinitely gentle - but even subtle prodding was enough to send pain jolting up her leg.

She stiffened. "...ouch!"

Immediately, the hands withdrew. "...My apologies, Director Hu. From what I've seen, your ankle has been badly sprained. I lack the resources to deal with it currently, especially considering where we are, so I suggest we bring you back to Liyue immediately so Dr. Baizhu can have a look."

Hu Tao cringed. "...ugh, that guy? Do we have to?"

"It's for the best, Director Hu." Hu Tao heard Zhongli shift beside her, the rustle of damp leaves under his shoes. His hands plucked at the jacket he draped over her, rearranging it tighter around her shoulders. "I'm sorry, but would you object to me carrying you?"

Hu Tao chuckled weakly. "...eh. Not like I can walk myself."

"...alright." He shifted again, and suddenly his face slid into view - stern but gentle, and infinitely familiar. For the first time in this disastrous night, Hu Tao finally felt like she was going to be okay. "I'm going to carry you on my back, if that's alright. When I turn, can you put your arms around my shoulders?"

Weakly, Hu Tao obliged; shuffling herself onto her knees and draping her arms over her consultant's shoulders. Zhongli gingerly settled his hands under her knees and hoisted her up, taking great care not to jostle her injured ankle. "...am I carrying you alright, Director? Are you experiencing any pain?"

Hu Tao shook her head from where it was nestled weakly against his shoulder. "...no. I'm fine."

"...very well. Then we shall head to Bubu Pharmacy immediately."

In a fit of sudden desperation, Hu Tao tightened her arms around his shoulders. "No. No, bring me to the Parlor instead. I want to-" I want to go home. "-I want to go back to Wangsheng."

Zhongli paused. "Director. Your ankle is badly sprained, and you've been stuck in the cold for hours. It's my belief that medical attention would be in your best interests."

Maybe so. But Hu Tao didn't want...she didn't want to go to an unfamiliar place, under the care of some strange doctor. She wanted the warmth of the Parlor, it's familiar walls, her second home. Yeye. Wo xiang nian ni. She shook her head stubbornly. "No. I want to go back to the Parlor. I just-" Her voice broke. She hated it when that happened. "Zhongli. Please."

Zhongli quieted. A soft silence followed, and Hu Tao feared that he would deny her the wish - but then she heard him sigh, long and drawn, and reaffirm his grip on her legs.

"...very well, Director. I'm sure Dr. Baizhu wouldn't object to a house call if we name the right price," he said. He sounded tired, but in those words was a gentleness she wasn't used to hearing. Not from him. "Don't fret, Hu Tao. I'll take you home."

Hu Tao nodded, closing her eyes. She felt warm against him, warm and cared for. The feeling of being held, borne aloft, carried by someone she trusted with her whole being...it was strangely familiar. Strangely nostalgic.

Yeye! Bao, bao!

Hao la, Xiao Tao.

She felt the breeze cut her cheeks as Zhongli ran through the woods, sprinting towards Liyue. She settled against him, feeling fatigue take its hold.

"...I wasn't sure you'd come," she mumbled, in a strange trance between sleep and wakefulness.

"Why would you think that?" she heard him say. There was a strange note in the cadence of his voice - but her mind wasn't clear enough to grasp what it was.

"...I haven't been very nice to you," she muttered, eyes fluttering shut. "...but you found me anyway. I shouldn't have doubted you. I trust you, Zhongli. I do."

A beat of silence. Hu Tao heard the wind whistle around her ears, the flap of Zhongli's coat around her shoulders. She closed her eyes and deafened the world.

When he spoke next, his voice was strangely soft. "...Hu Tao. Regardless of what you do, I wouldn’t just-"

She fell asleep before she heard his sentence.

*

I wouldn’t just abandon you.

Zhongli quieted, awaiting the young girl's response. When none was forthcoming, his brow furrowed, concerned. "...Hu Tao?"

He felt her settle against him, her breathing deep and even. At that, Zhongli felt some of his worry dissolve. Ah. Asleep, then. He supposed it was for the best - Director Hu had been stuck at the base of Wuwang Hill for hours, growing colder in the gathering dusk. As of now, she needed to conserve her strength.

Feeling the night grow darker around them, Zhongli hastened his pace. It was a long journey from here to the harbor, but still one he could easily brave.

*

When he finally slipped through the gates of Liyue, the skies were just beginning to lighten overhead. He slowed his pace to a jog, then winded smoothly around the buildings and stalls to arrive at the door of the Wangsheng funeral parlor.

As he walked up the steps, the Ferrylady was minding her post by the front of the building with a worried expression on her face. She looked up when she saw Zhongli - and jumped at the sight of the Director clinging limply to his back.

"All the Archons, Director Hu!" She exclaimed, stumbling down the steps towards them. Her hands hovered anxiously over the sleeping Director, trying to help but not quite knowing what to do. "Mr. Zhongli, what happened? Is she alright?"

"She's asleep right now. Could you please visit Bubu Pharmacy and ask Dr. Baizhu for a possible house call? And do tell him to prepare for a badly sprained ankle," he informed her, professional as ever, even as he shouldered open the front door of the Parlor and hastened his way inside.

Positioned in the main room was a small red bench, meant to seat waiting clients. Gingerly, Zhongli lowered Hu Tao onto said bench and rearranged the throw pillows to better accommodate her, pulling his coat more snugly around her frame. She stirred slightly, eyelids fluttering.

"...Zhongli?" she slurred, still half-asleep.

"...it's alright, Director Hu. We've made it back to Wangsheng," he said gently. "...please, continue to rest."

"...my hat?"

"Right here," Zhongli assuaged, settling the beloved hat onto her hands.

Hu Tao smiled. She wrapped her arms around the hat, head lolling in sleep, and when she looked at him her red eyes were warm like fire.

"...okay," she murmured. "...thank you, Zhongli."

Zhongli stared at her, feeling a curious warmth spread through his chest. Very gently, he reached out and smoothed a lock of hair from her face, hoping the familiarity of Wangsheng would help ease her from her injury.

"...you're welcome, Hu Tao," he said. """)

C:\Temp\ipykernel_11564\393573933.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(config.best_model_path, map_location=device))


(array(['categories:f/m', 'categories:gen', 'grief/mourning',
        'rating:general audiences', 'rating:teen and up audiences'],
       dtype=object),
 array([0.31319195, 0.7182171 , 0.30651104, 0.44916755, 0.60494334],
       dtype=float32))

In [19]:
def extract_cls_embeddings(model, dataloader, device):
    model.eval()

    all_embeddings = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting CLS embeddings"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            cls = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                return_embeddings=True
            )

            all_embeddings.append(cls.cpu())

    return torch.cat(all_embeddings, dim=0)

cls_model = BigBirdForTags(config.num_classes).to(device)
cls_model.load_state_dict(torch.load(config.best_model_path, map_location=device))

train_index_loader = DataLoader(
    train_ds,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0
)

test_index_loader = DataLoader(
    test_ds,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0
)

train_cls_embeddings = extract_cls_embeddings(
    model=cls_model,
    dataloader=train_index_loader,
    device=device
)

test_cls_embeddings = extract_cls_embeddings(
    model=cls_model,
    dataloader=test_index_loader,
    device=device
)

print("Train embeddings:", train_cls_embeddings.shape)
print("Test embeddings :", test_cls_embeddings.shape)

C:\Temp\ipykernel_24656\797227416.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cls_model.load_state_dict(torch.load(config.best_model_path, map_location=device))
Ext

KeyboardInterrupt: 

In [ ]:
train_cls_embeddings = torch.nn.functional.normalize(
    train_cls_embeddings, p=2, dim=1
)

test_cls_embeddings = torch.nn.functional.normalize(
    test_cls_embeddings, p=2, dim=1
)

In [ ]:
torch.save(
    {
        "train_embeddings": train_cls_embeddings,
        "test_embeddings": test_cls_embeddings,
        "model_name": config.model_name,
        "hidden_size": train_cls_embeddings.shape[1]
    },
    "cls_embeddings.pt"
)

In [24]:
query_text="""[TITLE] Vigilant Yaksha
[SUMMARY] Lumine lashes out in a desperate attempt to prove to Xiao she is a capable person.

(Universe where Aether chooses to abandon Lumine in Teyvat.)

This is the best birthday gift she could ever receive.
[FANDOMS] 原神 | Genshin Impact (Video Game)
[CHARACTERS]  Xiao | Alatus/Ying | Lumine (Genshin Impact)
[STORY] The air was clear and clean, and the moon hung low in the sky. Another sanctuary night in Liyue made possible by the vigilant yaksha.

The clouds seemed to thank him as they gleamed back at him from underneath his feet, bathed in purple light. The Qinxing flowers at the tops of the Minlin mountains bent back and forth in the gentle breeze; they brushed up against his fingers as if to hold him when nobody else could.

He sat on the edge of a grassy mountain, gazing out at the peaceful scene. Every breath he took was crisp and healing to his lungs.

These moments at the end of the day were what kept him going. Despite the immense pain in his body, he still managed to find some kind of peace as long as he sat amongst the Qinxing flowers.

Every day was the same, fight hard, suffer, then come to the haven of the mountain tops.

Today was different, though. He first felt it when the wind blew differently around him, as if one of the nearby updrafts were carrying something precious on its wings. Moments later, he heard the sound of grass crunching beneath careful feet and almost immediately recognized the presence as belonging to Lumine. He could hear her attempts to be quiet as she landed softly behind him, but they both knew he was aware whenever she was near.

What could have made her decide to visit him on this night? How had she found his hiding place? Her behavior would never make sense to him.

“Lumine?” he asked without turning his head.

If this disturbance had come from anyone else, he would’ve up and left without another word. People were not those he wished to grow close to.

But.... Lumine was different.

“Xiao! I thought I might find you here,” she said, walking up to the edge of the cliff.  “Do you mind if I join?”

“Is Paimon with you?” Xiao asked cryptically.

The girl let out that terribly lovely laugh he enjoyed so much. “No, I left her sleeping at Wangshu inn. She falls asleep way too quickly after she’s eaten.”

Xiao allowed himself to relax a little. The times when he could speak to the traveler without her talkative fairy were extremely few and far between. He cherished the precious seconds where the only thing he could hear was her voice.

He cleared his frivolous thoughts. “Good. She would have ruined the silence.”

Lumine covered her mouth quickly, as if suddenly self conscious. She tucked her hair behind her ear, a look of worry on her face.

The yaksha hesitated for a second or two, then patted the grass beside him in a gesture for her to join him.

Lumine obeyed wordlessly, carefully lowering herself to the ground beside him. She drew her knees up to her chest, looking almost somber as she perched herself above the majestic midnight view.

She wasn’t talking... why did she come if she didn’t intend to speak?

He finally forced himself to glance at her.

He felt a degree of fear when it came to looking at Lumine. It was mysteriously difficult to gaze on her for any short length of time.

Her silky blonde hair was always shining and flowing like a river of gold, and the urge to run his fingers through it was uncanny. Her eyes never stopped glittering like the wings of a crystalfly, and he found himself thinking he’d like to study them longer. Her skin was like soft powder, and when it was blemished with a wound, he wanted to press his hands to it and heal her with his own willpower.

He hated these feelings he’d been harboring for years on end. He knew it was what the humans called “love”, but he didn’t understand the stirring of his heart when she walked into the room, or the way he lost all focus when he spoke to her. He was supposed to be the vigilant yaksha, but when she was around he was exactly the opposite.

“Why do you remain so quiet?” he muttered huskily.

“... you said speaking would ruin the silence,” she replied.

Xiao mentally berated himself. He’d spoken carelessly about Paimon and her excessive noise. What he said about Paimon wasn’t intended to be applied to Lumine...

He shifted slightly, resting an arm on his knee. There was no rush to respond, but somehow his mind felt the urgent need to quell her fears. His heart was a terrible demon, racing against his ribcage-- an enemy to his calm.

“When you speak it is silence,“ Xiao told her, studying her face for a negative or positive response. “Your voice gives me the same peace that quiet offers me.”

Lumine’s drew her knees closer to her chest, throwing him a shy, sideways glance. “That makes me glad. I came here to ask you something,” she said, her spirits obviously boosted.

Xiao grunted wordlessly. There was no need for his voice to fill space she could be using.

“When we left the chasm, you mentioned destiny... and maybe that’s how our group came together and helped you learn what happened to Bosacius. Do you also believe that maybe it was destiny I came here? I’ve traveled every corner of Teyvat and my brother doesn’t want to see me, so now I’m trying to figure out why I’m still here and what my purpose is. Was I supposed to come here? Why?” her words were soaked in sadness, and her eyes seemed to droop with a tiredness he’d only seen in her after a long battle.

Xiao swallowed hard. To be perfectly honest he wasn’t sure why she was here. To him, she had appeared at first as a foreigner searching for her brother-- a mortal who was not worth his time. Now that he’d known her, helped her, and watched her struggle with her trials, he felt so drastically different about her.

There was much more to Lumine than her desire to be with her brother. There was strength, kindness, sadness, fear, worry, anger, all of which were mortal emotions in which he struggled to comprehend. He’d never wanted to understand them more than he did now.

“I used to feel the same,” he said quietly. “But ever since I fought with Bosacius, I think I am coming to learn why I survived. I... believe he would be satisfied to know I am here to protect the people of Liyue in his stead.”

He paused, forcing himself to look directly into the amber eyes that peered so kindly back at him. He felt a foreign sense of calm settle over him as he looked at her, the gentle night winds tugging at both their clothes. “The memory of your brother you carry with you-- I’m sure he would be satisfied with how well you have done.”

It was rare of Xiao to hand out praise (extremely rare) but he did it now for the one he admired the most. He knew that out of all the people he had met since his siblings died that Lumine would be the one they would have liked the most. She was hardworking, charitable, and gentle. Not only those things, but funny as well.

“You really think?” she mused, a tinge of sorrow in her voice as she pulled up some of the grass beside her. “I hope so, because when I think about the reason I started this whole journey and think about the way it’s ended...? It all feels pointless.”

“It wasn’t pointless,” Xiao blurted ungracefully.

You saved hundreds of thousands of people and spread good to every continent. You found your brother. It’s not your fault he didn’t want to be with you. That’s his loss.

He grabbed her shoulder with a strong hand, his eyes narrowing as he spoke. “Nothing you did was pointless. Do you hear me?” he berated sharply.

Lumine regarded him with startled eyes, a little gasp escaping her.

He pulled her closer upon instinct. “Just because your brother didn’t come back with you doesn’t make everything you did useless. Was helping me worthless?” The last question slipped out of his mouth without thought.

He quickly released her arm and looked away with embarrassment. “Forget it.  If you would like to think everything you have done is without use, then be that way.” He said with less gentleness than he’d intended.

For some reason he felt hurt.

Lumine took in a shaky breath and remained silent. He could feel her emotions wavering in the uncomfortable silence that fell between them. Normally, he would have simply vanished back to Wangshu Inn, but this time he felt almost obligated to sit beside the traveler. Again, he felt... hurt.  It was the same kind of hurt that plagued him each time he watched her leave to another region-- the same one that left him wondering: why doesn’t she stay?

He shook himself, angered by the confusing thoughts in his head.

“I shouldn’t have spoken so harshly,” he said with a heavy sigh. “I...”

She was covering her face with her hands again. He felt a strong negative aura from her body.

“Why are you covering your face?” he asked in a gentler tone, carefully reaching toward her.

He scooted closer, gently pulling her hands away from her face.

He was shocked to see that she was crying, tears streaming down her pretty cheeks, and her eyes twinkling with moisture.

Xiao himself had never cried before, but he’d seen others do it countless times throughout his lifetime. Crying was something mortals often did when they were distraught and hopeless.

“I’m sorry I made you feel like meeting you was pointless,” Lumine choked, sniffling miserably. “I didn’t mean that at all! I... I just meant that I didn’t bring my brother home and that’s what made it pointless-- but now I see that I was wrong. I’m sorry, Xiao.”

She stared up at him with a pitiful expression. “I didn’t know that I helped you.”

Xiao’s throat suddenly closed up a little. He had no idea what to say, he didn’t know what he was feeling. Seeing the way her lips quivered with sadness and her body shook with the same reflected feeling.  He knew she was genuinely distraught and grief stricken. The way he’d spoken to her felt terribly wrong.

Guilt spread like a wildfire. He didn’t know how to talk or relate to humans. He didn’t know how to be caring or empathetic! He didn’t know how to help her with something he struggled with every day of his life: grief. Her soul didn’t deserve such a punishment.

“Xiao, you’re crying,” Lumine whispered, tentatively reaching up for a moment before touching his face.

His brows furrowed with confusion. Crying? But I’ve never done that.

She brushed her thumb against his cheek, and sure enough: there were tears, and a lot of them.

Her hands were freezing.

“Your hands... they’re so cold,” he said absently, catching her fingers as she started to pull them away.

She peered at him in surprise as he took both her hands in his own, caressing them in an attempt to warm her. “You should tell me when you’re cold.”

“But...”

“Just let me help you,” he said, again, his voice coming harsher than intended.

“I’m okay,” she argued, throwing him that stubborn tone once again.

Lumine tried to pull her hands away, but Xiao pulled her toward him instead. “No, let me--“

“I’m fine, Xiao!” she yanked against him once again, as if the contact were making her uncomfortable.

The wind started to whistle around them harshly. A storm was certainly blowing in.

He forgot his strength as frustration boiled up inside. He pulled her forward, and she came crashing into his chest-- both of them teetering toward the edge of the cliff.  He instinctively embraced her and tensed in order to keep her from falling over the edge.

“This is why you should be more careful!” he scolded furiously over the rising wind. “I’m dangerous! I could’ve killed you just now! If you don’t even want my help then just go away!”

“NO!” she screamed, anguish escaping with the sound of her voice. “You feel like the closest thing I have to home!”

He scoffed bitterly, dry karma bleeding into his demeanor. “Me? Home? I’m a monster, Lumine.”

She clutched at the back of his shirt and took in a deep breath. Just as he began to wonder if maybe she’d given in, he felt her teeth suddenly clamp down on his shoulder.

Xiao cried out in alarm, tightening his grip on her. “What the he--“

She bit him harder, the groves of her teeth sinking deeper into skin. She dug her nails into his back and refused to let go, even as he tried to pry her away.

“Lumine! What are you doing!?” he struggled, eyes shut in pain as he sucked air through his grit teeth.

She finally released him from her iron bite, raising her head to look up at him with tear stained defiance. Pure determination was practically seeping out of her red faced expression. She looked like she’d been running on two hours of sleep, but somehow that made her more beautiful to him.

“Do regular people do that?” she demanded, her lip quivering. “I’m a monster, too, Xiao. We’re both monsters, see?”

She lunged forward to try and attack him again, but he threw up a hand to forestall her. “Stop! Stop! Wh-one second,” he gasped, grimacing with the shock of his burning shoulder.

He looked down, balking at the sheer size of the mark she’d left. It was actually bleeding... and extremely visible. The marks left by her teeth were pressed deep into his skin, overlapping with his green tattoo that flashed with adeptal energy. It was trying to alert him he was injured.

He glanced back down at Lumine who was still embracing him, tears falling down her face like a waterfall. She looked harmless, which was obviously wrong.

Confusion boggled his mind for several moments-- and then he finally understood. It clicked in his mind, and it felt as if the secrets of the girl were revealed to him.

“Listen,” he murmured, using the softest tone he could possibly muster. “I see what you’re trying to do-- but it won’t work. No matter how many times you bite me or yell at me, you will never be a monster like me.”

A fire lit up in her eyes, as if she’d silently accepted an unspoken challenge. Her lips fell into a deeper frown, and her gaze narrowed significantly.

He saw the look and hurriedly raised a silencing finger to her lips. They made eye contact, and slowly the flames of her resolution fizzled into cold embers.

“You didn’t want me to see your weaknesses earlier because you didn’t want me to feel you were incompetent? Is that correct?”

blushed hard, her shoulders slumping in defeat. “Yes... I just... I thought if you saw how weak I really am, you wouldn’t let me stay with you,” she said in a barely audible pitch.

Her words tingled in his heart like the vibrations of a lire string. “Stay.” “Stay with you” Had he heard that right? The wish of his since he’d met her all those years ago... could it truly be that she shared the same one? No. It couldn’t be. It didn’t make sense...

“You don’t have to prove yourself to me,” he told her, thoughts racing alongside his heart.

Lumine bit her lip with frustration, shaking from the cold and high emotions. “I don’t want you to think I’m a deadweight.”

“You have never been a deadweight,” Xiao argued, tightening his jaw with frustration.

“Yes, I have!”

“When?”

“When I was falling to my death and I couldn’t save myself!”

“You still saved Liyue...”

“Well there was the other time I tricked you into thinking I drown.”

“That doesn’t count.”

“But--“

“You have defended and saved Teyvat, Lumine. Why do you care about my opinion?”

She was stubborn-- always so stubborn.

Lumine huffed, her eyes brimming with tears again.

He had no idea how to handle her in such an emotional state. Being unfamiliar with emotions was a cosmically large disadvantage in this exact moment. His mind fumbled for ideas, something, anything to make her calm down. Her emotions were high, she was obviously tired, and her mood swings were becoming a little too difficult to handle.

He opened his mouth in an attempt to forestall her outburst. “I--“

“Why don’t you see? I just want you to love me!” she shouted, voice breaking alongside her fractured heart.

The words echoed off the sides of the mountain sides, reverberating around  in his skull, through his bones and down into his heart.

He stared blankly as she pushed herself away and covered her eyes, sobbing into her hands like a lost child. This side of Lumine was so volatile, shy, sad, and filled with anguish. It had always been there, hidden behind her polished exterior-- but here in the quiet land of the yaksha had emerged her insides, cut loose from losing her brother. The world had crashed down for her.

“I’ve spent so long just feeling this way from a distance,” she cried, words muffled. “Every time I left the inn, I always regretted not just telling you how I felt! I thought about quitting my journey so many times so I could stay there in the treehouse. But I chose to keep going only to be abandoned by my brother! Now my journey is over, and Paimon is leaving to go far away. I came here to ask you if I could stay at the inn, but my emotions got the better of me. I always hoped you felt the same, but I know that you don’t and you never did. I’m so stupid! I’m so stupid!”

She was so wrong. She was so wrong.

Lumine stood up, angrily dashing the tears from her eyes. “Bye, Xiao.”

She started to turn around, but the yaksha caught her by the wrist. She looked down at him with exhausted eyes, and he stared up at her with fresh ones.

The clouds hovered around them like guardian angels, stars twinkling as  if whispering to one another. The wind died like the last note on a violin. All was quiet as he squeezed her hand and caught his breath.

“I love you,” he admitted, voice ringing clear through the silent mountains.

The three words were so simple, so short, like all his other sentences... but so different from anything he had ever said before. They felt so right, clean, clear--gentle.

His nerves were high, vision unsteady, karma shaking, heart pounding, cheeks flushing. How did just her presence do this to him?

Lumine suddenly gasped and her knees gave out underneath her. Xiao leapt to his feet, grabbing onto her shoulders and carefully guiding her to the ground.

Both of them were trembling as they clung to each other.

They sat down on the grass again, Xiao carefully sliding his hands down her elbows as he knelt in front of her. “There... just sit still,” he breathed heavily, nerves high from his confession.

She looked at him with wide, star struck eyes. She looked so beautiful with that tired auburn gaze, mouth slightly ajar in shock.

“You do...?” she finally asked, her line of sight darting all over his face-- as if searching to find if this were a dream or not.

Xiao nodded, swallowing hard with embarrassment as he adjusted himself to sit criss cross. A lump rose up in his throat, and it had never been so hard to breathe. “I always have.” It felt surreal to speak those words.

Lumine’s hands rested limply on the grass, her breath escaping in little bursts of cold plumes, her cheeks shining with blush and tears.

She had never looked so beautiful.

The traveler reached forward hesitantly, her arms outstretched in a feeble attempt to embrace him.

Without needing a second hint, Xiao immediately took her up in his arms and cradled her in his lap. Her head came against his chest, and his strong arms supported her cold, delicate frame.

It felt terribly indulgent to hold her at such a close proximity. He knew it was wrong to allow her so close to his karma inflicted vessel, but it felt so unequivocally right at the same time. Her presence took away his pain, and her touch eased his turmoil. She had never shown signs of being affected by his darkness before, and finally, in this moment, he allowed himself to believe she would be alright to be this close to him.

Her hand pressed up against his chest, causing a shiver to run down his spine as he found himself enjoying the foreign close contact

She stared up at him with those eyes again, and now--even if he wanted to-- he wouldn’t be able to look away.

“I love you, Xiao,” she whispered.

The words leaving her mouth sent a tidal wave of emotions cascading through his body. He felt himself shudder as the weight of her words crashed into him again. He already knew she loved him from her earlier outburst, but hearing it so clearly directed at him sent his mind reeling.

“Why?” was all he could ask. Why would you set your heart on someone like me?

She stayed silent for a long while, struggling with her emotions for a little.

“You’re gentle with me, and you never want me to be in danger. Everywhere else I go, they put me straight into the fight. You’re the only one I feel like I can relax around. Even though your attitude is distant on the outside, when I talk with you I see how much you really care. I know you tell yourself lies to make things easier, but I see how much you love Liyue and the people in it,” she explained. “You’re selfless, and you’re protective like how my brother used to be. I can defend myself, but... I want to be taken care of again, like how I used to be taken care of.”

Her eyes welled up with tears again, and she turned her gaze away from him. “I guess... those are a few reasons I feel the way I do.”

Xiao gently touched her face, carefully turning her head back toward him. The thought of being someone’s safe place was foreign idea, but not an unwelcome one.

“You should have said that from the start,” he said, a rare smirk ghosting at his lips as he glanced pointedly at his bleeding shoulder. “You have no respect for the adepti.”

Lumine blushed profusely, smacking him with the back of her hand. “Don’t say that!”

To both of their surprise, he actually allowed a little chuckle to escape him--eyes twinkling with amusement at her mood swings.  “It’s fine.... I don’t need respect from you, anyway.”

Lumine hurriedly wiped the shocked expression off her face, crossing her arms across her chest stubbornly.

“You are... so difficult to comprehend,” he said hesitantly, shaking his head. “But I would be lying to say I didn’t want to accept your request.”

“I can stay with you?” she asked a little doubtfully, obviously worried to get her hopes up.

Xiao nodded calmly, his eyes lingering on her as he took in her features. The air grew silent around them as he formulated his next words. What could he say to her? This was new territory for him.

“It’s... already too late to sever the connection I have to you. So, as long as you live, you are welcome to stay at Wangshu Inn with me.”

Lumine expression relaxed into pure relief. She let out a huge sigh, tension releasing from her body as she quickly wrapped her arms around his neck and embraced him.

Xiao hugged her back, still partially shocked by his decision to let her stay. Hadn’t this been his wish all along? The wish for her to never leave? Why was it so shocking to see his desire fulfilled?

“Will you leave after a while?” he asked hesitantly, his arms tightening around her slim form.

The thought of watching her leave him again as he stood on the balcony of the inn burned a hole inside him. The reminder of his heart aching as he watched her disappear over the horizon-- wondering if she would live till their next meeting; the feeling of worry when he saw her new battle scars each time she returned-- loathing the fact that she hadn’t called his name. He remembered the ever present dread that she would find her brother and leave for good. He had been dangerously attached to her for far too long.

“No.” Her answer came without hesitation. “I want to stay with you forever.”

Xiao’s chest constricted, and those strange tears formed in his eyes once more. His voice choked inside his throat, and for a moment, he found himself unable to speak.

He took in a weak breath, allowing himself to close his eyes for a moment. Just for a few minutes... maybe he didn’t have to be the vigilant yaksha. He brought himself closer to her, the contact healing wounds he didn’t know he had. The pain of being alone for so long felt as if it were slowly being cured by her close proximity.

He took in a quiet breath, then pressed his lips against her soft, bare shoulder. It was warm, like the presence of fire on a winter’s night. His own shoulder dripped with hot blood from earlier, but in that moment, he was completely unaware. Nothing else mattered, not the demons, the pain of the past, or anxieties about the future-- only her arms around him and her words on his soul. He didn’t have to vigilant anymore, not right now.

He carefully drew his lips away from her skin, turning his head inward to rest. He still didn’t know why he was the only survivor out of all his siblings, or why Lumine was unable to save her brother-- but he knew this was where he wanted to be for the rest of his life.

Soft wind, cold air and a warm embrace. He nuzzled closer, whispering the words he vowed to eternally live by:

“I will be here forever... if that’s as long as you wish.” """

In [25]:
def neural_search_ao3(
    query_text: str,
    model: nn.Module,
    tokenizer,
    embeddings: torch.Tensor,
    dataset: pd.DataFrame,
    mlb: MultiLabelBinarizer,
    top_k: int = 5,
    device='cuda'
):
    model.eval()
    
    encoding = tokenizer(
        query_text,
        truncation=True,
        padding='max_length',
        max_length=config.max_length,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        query_emb = model(input_ids, attention_mask, return_embeddings=True)
        query_emb = torch.nn.functional.normalize(query_emb, p=2, dim=1)
    
    sims = cosine_similarity(query_emb.cpu().numpy(), embeddings.cpu().numpy())[0]
    
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    
    tag_indices = np.where(probs > 0.40)[0]
    query_tags = set([mlb.classes_[i] for i in tag_indices])

    def tag_score(work_tags):
        return len(query_tags.intersection(work_tags))

    results = []
    for idx, sim in enumerate(sims):
        work_tags = set(dataset.iloc[idx]["all_tags_normalized"])
        score = sim * 2 + tag_score(work_tags)
        results.append((idx, score, sim))
    
    results = sorted(results, key=lambda x: x[1], reverse=True)
    top_results = []
    
    for idx, score, sim in results[:top_k]:
        row = dataset.iloc[idx]
        top_results.append({
            'title': row['title'],
            'tags': row['all_tags_normalized'],
            'fandoms': row.get('fandoms', []),
            'categories': row.get('categories', []),
            'relationships': row.get('relationships', []),
            'cosine_sim': float(sim),
            'score': float(score),
            'work_id': row['work_id']
        })
    
    return top_results

In [26]:
def print_search_results(results):
    for i, r in enumerate(results, 1):
        work_url = f"https://archiveofourown.org/works/{r['work_id']}"
        print(f"=== {i}. {r['title']} ===")
        print("Fandoms:", ", ".join(r.get('fandoms', [])))
        print("Relationships / Pairings:", ", ".join(r.get('relationships', [])))
        print("Categories:", ", ".join(r.get('categories', [])))
        print("Tags:", ", ".join(r['tags']))
        print(f"Cosine similarity: {r['cosine_sim']:.3f}")
        print(f"Combined score: {r['score']:.3f}")
        print("Link:", work_url)
        print("-" * 60)

In [27]:
predict(query_text)

(array(['angst', 'categories:f/m', 'drama', 'rating:mature',
        'rating:teen and up audiences'], dtype=object),
 array([0.41799316, 0.6401293 , 0.41438535, 0.4212067 , 0.49411523],
       dtype=float32))

In [17]:
results = neural_search_ao3(
    query_text=query_text,
    model=cls_model,
    tokenizer=tokenizer,
    embeddings=train_cls_embeddings,
    dataset=df_filtered,
    mlb=mlb,
    top_k=5,
    device=device
)

print_search_results(results)

NameError: name 'train_cls_embeddings' is not defined

Ещё можно добавить функцию, чтобы в зависимости от уверенности модели в теге увеличивался вес тега